In [8]:
from scipy.spatial import ConvexHull
import numpy as np
import os
import cv2 as cv
import open3d as o3d
from matplotlib import pyplot as plt
from scipy.io import savemat
import glob

In [9]:
checkerboard = "patients\case_4\day_1\calib\scene_1\photo.jpg"
checkerboardDepth = "patients\case_4\day_1\calib\scene_1\depth_camera\depth_camera_1.dat"
wound = "patients\case_4\day_1\data\scene_1\photo.jpg"
woundMask = "patients\case_4\day_1\data\scene_1\mask.png"
woundDepth = "patients\case_4\day_1\data\scene_1\depth_camera\depth_camera_1.dat"

In [10]:
# Defining the dimensions of checkerboard
CHECKERBOARD = (7,8)
criteria = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None
# Arrays to store object points and image points from all the images.
objpoints = [] # 3d point in real world space

imgpoints = [] # 2d points in image plane.

file_pattern = "photo.jpg"

root_directory = "patients"

# Use glob to find files matching the pattern in subdirectories
images = glob.glob(f"{root_directory}/**/calib/**/{file_pattern}", recursive=True)
images = images[0:8] #No significant accuracy gains with > 10 calibration images
for fname in images:
    img = cv.imread(fname)
    #down_width = 176
    #down_height = 144
    #down_points = (down_width, down_height)
    #img = cv.resize(img, down_points, interpolation= cv.INTER_LINEAR)
    gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv.findChessboardCorners(gray, CHECKERBOARD, cv.CALIB_CB_ADAPTIVE_THRESH + cv.CALIB_CB_FAST_CHECK + cv.CALIB_CB_NORMALIZE_IMAGE)
    if ret == True:
        objpoints.append(objp)
        
        corners2 = cv.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
        #corners2[:,:,0] = corners2[:,:,0] * 176 / 4896
        #corners2[:,:,1] = corners2[:,:,1] * 144 / 3264
        imgpoints.append(corners2)
 
        img = cv.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
     
    #cv.imshow('img',img)
    #cv.waitKey(0)
 
cv.destroyAllWindows()
print(gray.shape[::-1])
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)

(4896, 3264)


In [ ]:
print(ret) # Objective function value
print(mtx)    # Camera matrix
print(dist)   # Distortion coefficients

In [13]:
##Scale factor applied to convert from png resolution to depth sensor
mtx[0,0] = mtx[0,0] * 144/ 3264
mtx[1,1] = mtx[1,1]* 144/ 3264
#mtx[0,2] = 144/2
#mtx[1,2] = 176/2

In [8]:
print(mtx)

[[7.56430355e+00 0.00000000e+00 2.26793937e+03]
 [0.00000000e+00 7.55689240e+00 1.36267939e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [14]:
##Point Cloud Plotting / Hull construction
depthMap = np.loadtxt(woundDepth, comments='%')
mask = cv.imread(woundMask)
maskDown = cv.resize(mask, (176, 144))
print(maskDown.shape)
image = cv.imread(wound)
#depthMap = (depthMap[0:144,:] + depthMap[144:288, :] + depthMap[288:432, :] + depthMap[432:576, :] + depthMap[576:720, :])/5
depthMap = depthMap[432:576, :]
#depthMap[maskDown[:,:,0] == 0] = 0
#intrinsic = o3d.camera.PinholeCameraParameters(mtx)
depth_image_o3d = o3d.geometry.Image(depthMap.astype(np.float32))
o3d_camera_intrinsic = o3d.camera.PinholeCameraIntrinsic(4896, 3264, 
                                                          mtx[0,0], mtx[1,1], 
                                                          mtx[0,2] * 144/ 3264, mtx[1,2] * 176/4896)
pointCloud = o3d.geometry.PointCloud.create_from_depth_image(depth_image_o3d, o3d_camera_intrinsic)
#print(pointCloud.points)
#cl, ind = pointCloud.remove_radius_outlier(nb_points= 1, radius=50)
#inlier_cloud = cl.select_by_index(ind)
#alpha = .03
#mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
    #pointCloud, alpha)
#mesh.compute_vertex_normals()
hull = ConvexHull(np.asarray(pointCloud.points))
print(hull.area)
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='pointcloud', width=3000, height=2000)
vis.add_geometry(pointCloud)
vis.run()
vis.destroy_window()
#mdic = {"a": pointCloud, "label": "experiment"}
#savemat("woundPC.mat", mdic)


(144, 176, 3)
13703874800.272175


In [ ]:
##Internal Plotting
fig = plt.figure()
ax = fig.add_subplot(111,projection='3d')
ax.scatter(pointCloud[:,:,0], pointCloud[:,:,1], pointCloud[:,:,2])
plt.show()

In [ ]:
##Debugging
input_file = "depth-mesh.ply"
pcd = o3d.io.read_point_cloud(input_file) # Read the point cloud
point_cloud_in_numpy = np.asarray(pcd.points) 
point_cloud = point_cloud_in_numpy.reshape((144, 176, 3))
x_new = np.multiply(point_cloud[:,:,0],mask)
y_new = np.multiply(point_cloud[:,:,1],mask)
z_new = np.multiply(point_cloud[:,:,2],mask)
mdic = {"a": mask, "label": "experiment"}
savemat("woundPC.mat", mdic)

In [ ]:
##Extra function used to sanity check point cloud construction
def depth_map_to_point_cloud(depth_map, camera_matrix, mask_array=None):
    # Get depth map shape
    
    height, width = depth_map.shape
    print(depth_map.shape)
    # Create 2D pixel grid
    u, v = np.meshgrid(np.arange(width), np.arange(height))

    # Homogeneous coordinates
    homogenous_coords = np.vstack((u.flatten(), v.flatten(), np.ones_like(depth_map.flatten())))
    # Inverse of the camera matrix
    inv_camera_matrix = np.linalg.inv(camera_matrix)

    # Transform pixel coordinates to normalized image coordinates
    normalized_coords = np.dot(camera_matrix, homogenous_coords)

    # Convert normalized image coordinates and depth to 3D coordinates
    z_3d = depth_map.flatten()
    #x_3d = normalized_coords[0] * z_3d
    y_3d = normalized_coords[1] * z_3d
    x_3d = (homogenous_coords[0] - camera_matrix[0,2]) * ( depth_map.flatten() / camera_matrix[0,0])
    #y_3d = (homogenous_coords[1] - camera_matrix[1,2]) * ( depth_map.flatten() / camera_matrix[1,1])
    #z_3d = depth_map.flatten()
    # Stack 3D coordinates to form the point cloud
    point_cloud = np.stack((x_3d, y_3d, z_3d), axis=-1)
    print(point_cloud)
    point_cloud = o3d.geometry.PointCloud.create_from_depth_image(depth_map, camera_matrix)
    print(point_cloud)
    if mask_array is not None:
            point_cloud = np.ma.masked_where(mask_array[:,:,0] == 0, point_cloud)
    
    return point_cloud.reshape((height, width, 3))

In [ ]:
def center_scale(img, dim,resize): 
    """Returns center cropped image Args: img: image to be center cropped dim: dimensions (width, height) to be cropped """ 
    width, height = img.shape[1], img.shape[0] # process crop width and height for max available dimension 
    crop_width = dim[0]
    crop_height = dim[1]
    mid_x, mid_y = int(width/2), int(height/2) 
    cw2, ch2 = int(crop_width/2), int(crop_height/2) 
    crop_img = img[mid_y-ch2:mid_y+ch2, mid_x-cw2:mid_x+cw2] 
    cv.resize(mask, (512,512), interpolation= cv.INTER_LINEAR)
    return crop_img

In [ ]:
def mkdirs(newdir,mode=0o777):
    try:
        os.makedirs(newdir, mode)
    except OSError as err:
        return err

In [ ]:
import shutil
image_pattern = "photo.jpg"
mask_pattern = "mask.png"
# Specify the root directory where you want to search
root_directory = "patients"

# Use glob to find files matching the pattern in subdirectories
print(f"../{root_directory}/**/data/**/{image_pattern}")
images = glob.glob(f"{root_directory}/**/data/**/{image_pattern}", recursive=True)
masks = glob.glob(f"{root_directory}/**/data/**/{mask_pattern}", recursive=True)
print(masks)
maskDest = "HarDNet-MSEG-master/lib/TrainDataset/masksW"
mkdirs(maskDest)
imgDest = "HarDNet-MSEG-master/lib/TrainDataset/imagesW"
mkdirs(imgDest)
for i in range(len(images)):
    dest = maskDest + "/entry" + str(i)
    print(dest)
    shutil.copyfile(masks[i], maskDest)
    dest = imgDest + "/entry" + str(i)
    print(dest)
    shutil.copyfile(images[i], imgDest)